 # File zum speichern der Locations von D_p
 

In [1]:
import numpy as np

In [2]:
# Konstanten Einheitenlos
M_STAR=1
mmr=2

G=1
R_0=1 #1 au

SIGMA_0=1e-4*M_STAR *R_0**(-2)
SIGMASLOPE=1.5

ALPHAVISCOSITY=1e-3       
TSLOPE=0.5           
FLARINGINDEX=0.25        
H_0=0.06

In [3]:
#Functions for the Resonance location
def omega_kepler(r):
    return np.sqrt(G*M_STAR/(r**3))

def surface_density(r):
    return SIGMA_0* (r/R_0)**(-SIGMASLOPE)
def aspect_ratio(r):
    return H_0 *(r/R_0)**FLARINGINDEX

def r_lim_k(r):
    return r-aspect_ratio(r)*r *2/3 ,r+aspect_ratio(r)*r *2/3



def vk(r):
    return np.sqrt(G*M_STAR/r)

def cs(r):
    return aspect_ratio(r)*vk(r)

def press(r):
    return surface_density(r)*cs(r)**2

def dfdlnr(f,r,*args):
    rp       = r * (1+1e-5)
    lnr      = np.log(r)
    lnrp     = np.log(rp)
    return (f(rp,*args)-f(r,*args))/(lnrp-lnr)
                                     
def dlnsigmadlnr(r):
    return dfdlnr(sigma,r)/sigma(r)
                                     
def dlnpdlnr(r):
    return dfdlnr(press,r)/press(r)
                                     
def omphi(r):
    return omega_kepler(r) * np.sqrt(1+(cs(r)**2/vk(r)**2)*dlnpdlnr(r))
                                     
def kappa(r):
    qkap = -dfdlnr(omphi,r)/omphi(r)
    return np.sqrt(2*(2-qkap))*omphi(r)
                                     

In [4]:
#Resonance locations
def D10_pos2(rp,m, nr=1000000):
    r=np.logspace(-1,1.3, nr)

    D10=kappa(r)**2-m[:,None][:,None]**2 *(omphi(r)-omega_kepler(rp)[:,None])**2
    idx, idy, idz = np.where(np.sign(D10[:,:,1:]*D10[:,:,:-1])==-1)
    
    
    idxin , idyin, idzin = idx[::2],idy[::2],idz[::2]
    idxout, idyout, idzout = idx[1::2],idy[1::2],idz[1::2]
    
    posout = np.zeros_like(D10[:,:, 0])
    posin = np.zeros_like(D10[:,:, 0])
    
    if m==np.arange(1,2):   
        
        posin[0]=0
        posout[idx, idy]=r[idz]
        
    else:
        posin[idxin, idyin] = (r[idzin]+r[idzin+1])/2
        posout[idxout, idyout] = (r[idzout]+r[idzout+1])/2    
                                            
    return posin, posout 


''

In [5]:
#Abspeichern der Daten

with open(f"D_p_in{mmr}_{H_0}_{FLARINGINDEX}_{SIGMA_0}.dat",'a') as f:
    with open(f"D_p_out{mmr}_{H_0}_{FLARINGINDEX}_{SIGMA_0}.dat",'a') as g:
        for j in np.arange(1,20):
            
            k_array_in=np.array([])
            k_array_out=np.array([])
            
            
            for i in np.arange(0.1,2,0.1):

                rp=np.arange(1+i,1.1+i-0.001,0.001)
                rpo=rp*mmr**(2/3)
                m=np.arange(j,j+1)
                ki=D10_pos2(rp,m, nr=1000000)[0]
                k0=D10_pos2(rpo,m, nr=1000000)[1]

                k_array_in =np.append(k_array_in,*ki)
                k_array_out =np.append(k_array_out,*k0)


            np.savetxt(f,k_array_in.flatten(), newline = " ")
            np.savetxt(g,k_array_out.flatten(), newline = " ")


a=np.loadtxt(f"D10_in{mmr}.dat")
a.reshape(len(np.arange(1,20)),len(np.arange(1.1,2.3,0.001)))